# EddyAI Project

In [35]:
import pandas as pd
import numpy as np

In [76]:
df = pd.read_csv("/Users/nielsenreaveley/downloads/edubasealldata20250224.csv", encoding='Windows-1252', low_memory=False)
df.head()

,URN,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentTypeGroup (code),EstablishmentTypeGroup (name),EstablishmentStatus (code),EstablishmentStatus (name),ReasonEstablishmentOpened (code),ReasonEstablishmentOpened (name),OpenDate,ReasonEstablishmentClosed (code),ReasonEstablishmentClosed (name),CloseDate,PhaseOfEducation (code),PhaseOfEducation (name),StatutoryLowAge,StatutoryHighAge,Boarders (code),Boarders (name),NurseryProvision (name),OfficialSixthForm (code),OfficialSixthForm (name),Gender (code),Gender (name),ReligiousCharacter (code),ReligiousCharacter (name),ReligiousEthos (name),Diocese (code),Diocese (name),AdmissionsPolicy (code),AdmissionsPolicy (name),SchoolCapacity,SpecialClasses (code),SpecialClasses (name),CensusDate,NumberOfPupils,NumberOfBoys,NumberOfGirls,PercentageFSM,TrustSchoolFlag (code),TrustSchoolFlag (name),Trusts (code),Trusts (name),SchoolSponsorFlag (name),SchoolSponsors (name),FederationFlag (name),Federations (code),Federations (name),UKPRN,FEHEIdentifier,FurtherEducationType (name),LastChangedDate,Street,Locality,Address3,Town,County (name),Postcode,SchoolWebsite,TelephoneNum,HeadTitle (name),HeadFirstName,HeadLastName,HeadPreferredJobTitle,BSOInspectorateName (name),InspectorateReport,DateOfLastInspectionVisit,NextInspectionVisit,TeenMoth (name),TeenMothPlaces,CCF (name),SENPRU (name),EBD (name),PlacesPRU,FTProv (name),EdByOther (name),Section41Approved (name),SEN1 (name),SEN2 (name),SEN3 (name),SEN4 (name),SEN5 (name),SEN6 (name),SEN7 (name),SEN8 (name),SEN9 (name),SEN10 (name),SEN11 (name),SEN12 (name),SEN13 (name),TypeOfResourcedProvision (name),ResourcedProvisionOnRoll,ResourcedProvisionCapacity,SenUnitOnRoll,SenUnitCapacity,GOR (code),GOR (name),DistrictAdministrative (code),DistrictAdministrative (name),AdministrativeWard (code),AdministrativeWard (name),ParliamentaryConstituency (code),ParliamentaryConstituency (name),UrbanRural (code),UrbanRural (name),GSSLACode (name),Easting,Northing,MSOA (name),LSOA (name),InspectorateName (name),SENStat,SENNoStat,BoardingEstablishment (name),PropsName,PreviousLA (code),PreviousLA (name),PreviousEstablishmentNumber,Country (name),UPRN,SiteName,QABName (code),QABName (name),EstablishmentAccredited (code),EstablishmentAccredited (name),QABReport,CHNumber,MSOA (code),LSOA (code),FSM,AccreditationExpiryDate
0,100000,201,City of London,3614.0,The Aldgate School,2,Voluntary aided school,4,Local authority maintained schools,1,Open,0,Not applicable,NaN,0,Not applicable,NaN,2,Primary,3.0,11.0,1,No boarders,Has Nursery Classes,2,Does not have a sixth form,3,Mixed,2,Church of England,Does not apply,CE23,Diocese of London,0,Not applicable,271.0,2,No Special Classes,18-01-2024,244.0,132.0,112.0,21.7,0,Not applicable,NaN,NaN,Not applicable,NaN,Not under a federation,NaN,NaN,10079319.0,NaN,Not applicable,24-01-2025,St James's Passage,Duke's Place,NaN,London,NaN,EC3A 5DE,www.thealdgateschool.org,2.072831e+09,Miss,Alexandra,Allan,Headteacher,Not applicable,NaN,NaN,NaN,Not applicable,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,Not applicable,Not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,London,E09000001,City of London,E05009308,Portsoken,E14001172,Cities of London and Westminster,A1,(England/Wales) Urban major conurbation,E09000001,533498.0,181201.0,City of London 001,City of London 001F,NaN,NaN,NaN,NaN,NaN,999,NaN,NaN,NaN,2.000001e+11,NaN,0,Not applicable,0,Not applicable,NaN,NaN,E02000001,E01032739,53.0,NaN
1,100001,201,City of London,6005.0,City of London School for Girls,11,Other independent school,3,Independent schools,1,Open,0,Not applicable,01-01-1920,0,Not applicable,NaN,0,Not applicable,7.0,18.0,1,No boarders,No Nursery Classes,1,Has a sixth form,2,Girls,6,None,Inter- / non- denominational,0000,Not applicable,2,Selective,860.0,2,No Special Classes,18-01-2024,764.0,0.0,764.0,NaN,0,Not applicable,NaN,NaN,Not applicable,NaN,Not appl

In [77]:
# ensures all columns are shown without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  # prevents column wrapping
pd.set_option('display.max_colwidth', None)  # ensures full content in each column

In [39]:
df.shape

(51564, 135)

 We have 51,564 rows of data, and 135 columns. 

In [41]:
df["PhaseOfEducation (name)"].value_counts()

Primary                    30066
Not applicable             12274
Secondary                   6932
Middle deemed secondary      630
Nursery                      619
16 plus                      576
Middle deemed primary        263
All-through                  204
Name: PhaseOfEducation (name), dtype: int64

The below code cell shows the large majority of "Not Applicable" schools for "PhaseOfEducation" are below secondary education. This is because the large majority of schools in "Not Applicable" category have students with minimum age of below 11, so for now we'll leave "Not Applicable" schools out.

In [43]:
filtered_df = df.loc[
    (df["PhaseOfEducation (name)"] == "Not applicable") & 
    (df["StatutoryLowAge"] > 10)
]

count = len(filtered_df)  # Get the number of matching rows
print(count)

2728


In [44]:
# this shows a massive amount of schools have closed or are due to close, so we can disregard them in below function.
df["EstablishmentStatus (name)"].value_counts()

Open                           27119
Closed                         24363
Open, but proposed to close       43
Proposed to open                  39
Name: EstablishmentStatus (name), dtype: int64

# Step 1: Creating Python script to filter target audience (secondary pupils and above)

In [78]:
def filter_schools(dataframe):
    # function below deletes everything included inside parenthesis, and leaves other values remaining in dataframe. 
    return dataframe[
        ~dataframe["PhaseOfEducation (name)"].isin(["Primary", "Not applicable", "Nursery", "Middle deemed primary"]) &
        ~dataframe["EstablishmentStatus (name)"].isin(["Closed", "Open, but proposed to close"])]

revised_df = filter_schools(df) 
        

In [46]:
revised_df["PhaseOfEducation (name)"].value_counts()

Secondary                  3171
16 plus                     296
All-through                 163
Middle deemed secondary      87
Name: PhaseOfEducation (name), dtype: int64

As shown above, we have 3,717 schools that we can definetley market our product to.

Orignally, the above function only filtered for the school being secondary school and above, which yielded around 8,342 schools, but after further filtering for schools that are closed or due to closed, we're left with 3,717. 

# Step 2: Build Python tool to find emails for target schools, digging to depth of 5 on their websites

website column is called **SchoolWebsite**

In [47]:
revised_df.head()

,URN,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentTypeGroup (code),EstablishmentTypeGroup (name),EstablishmentStatus (code),EstablishmentStatus (name),ReasonEstablishmentOpened (code),ReasonEstablishmentOpened (name),OpenDate,ReasonEstablishmentClosed (code),ReasonEstablishmentClosed (name),CloseDate,PhaseOfEducation (code),PhaseOfEducation (name),StatutoryLowAge,StatutoryHighAge,Boarders (code),Boarders (name),NurseryProvision (name),OfficialSixthForm (code),OfficialSixthForm (name),Gender (code),Gender (name),ReligiousCharacter (code),ReligiousCharacter (name),ReligiousEthos (name),Diocese (code),Diocese (name),AdmissionsPolicy (code),AdmissionsPolicy (name),SchoolCapacity,SpecialClasses (code),SpecialClasses (name),CensusDate,NumberOfPupils,NumberOfBoys,NumberOfGirls,PercentageFSM,TrustSchoolFlag (code),TrustSchoolFlag (name),Trusts (code),Trusts (name),SchoolSponsorFlag (name),SchoolSponsors (name),FederationFlag (name),Federations (code),Federations (name),UKPRN,FEHEIdentifier,FurtherEducationType (name),LastChangedDate,Street,Locality,Address3,Town,County (name),Postcode,SchoolWebsite,TelephoneNum,HeadTitle (name),HeadFirstName,HeadLastName,HeadPreferredJobTitle,BSOInspectorateName (name),InspectorateReport,DateOfLastInspectionVisit,NextInspectionVisit,TeenMoth (name),TeenMothPlaces,CCF (name),SENPRU (name),EBD (name),PlacesPRU,FTProv (name),EdByOther (name),Section41Approved (name),SEN1 (name),SEN2 (name),SEN3 (name),SEN4 (name),SEN5 (name),SEN6 (name),SEN7 (name),SEN8 (name),SEN9 (name),SEN10 (name),SEN11 (name),SEN12 (name),SEN13 (name),TypeOfResourcedProvision (name),ResourcedProvisionOnRoll,ResourcedProvisionCapacity,SenUnitOnRoll,SenUnitCapacity,GOR (code),GOR (name),DistrictAdministrative (code),DistrictAdministrative (name),AdministrativeWard (code),AdministrativeWard (name),ParliamentaryConstituency (code),ParliamentaryConstituency (name),UrbanRural (code),UrbanRural (name),GSSLACode (name),Easting,Northing,MSOA (name),LSOA (name),InspectorateName (name),SENStat,SENNoStat,BoardingEstablishment (name),PropsName,PreviousLA (code),PreviousLA (name),PreviousEstablishmentNumber,Country (name),UPRN,SiteName,QABName (code),QABName (name),EstablishmentAccredited (code),EstablishmentAccredited (name),QABReport,CHNumber,MSOA (code),LSOA (code),FSM,AccreditationExpiryDate
49,100049,202,Camden,4104.0,Haverstock School,1,Community school,4,Local authority maintained schools,1,Open,0,Not applicable,NaN,0,Not applicable,NaN,4,Secondary,11.0,18.0,1,No boarders,No Nursery Classes,1,Has a sixth form,3,Mixed,0,Does not apply,Does not apply,0000,Not applicable,4,Non-selective,1336.0,2,No Special Classes,18-01-2024,973.0,554.0,419.0,67.5,0,Not applicable,NaN,NaN,Not applicable,NaN,Supported by a federation,17669,H3 Federation,10002938.0,NaN,Not applicable,15-11-2024,24 Haverstock Hill,Chalk Farm,NaN,London,NaN,NW3 2BQ,http://www.haverstock.camden.sch.uk/,2.072671e+09,Ms,Katie,Metselaar,Headteacher,Not applicable,NaN,NaN,NaN,Not applicable,NaN,Not applicable,Not applicable,Not applicable,NaN,NaN,Not applicable,Not applicable,ASD - Autistic Spectrum Disorder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Resourced provision,8.0,8.0,NaN,NaN,H,London,E09000007,Camden,E05013660,Haverstock,E14001290,Holborn and St Pancras,A1,(England/Wales) Urban major conurbation,E09000007,528159.0,184498.0,Camden 012,Camden 012B,NaN,NaN,NaN,NaN,NaN,999,NaN,NaN,NaN,1.009182e+10,NaN,0,Not applicable,0,Not applicable,NaN,NaN,E02000177,E01000902,461.0,NaN
50,100050,202,Camden,4166.0,Parliament Hill School,1,Community school,4,Local authority maintained schools,1,Open,0,Not applicable,NaN,0,Not applicable,NaN,4,Secondary,11.0,18.0,1,No boarders,No Nursery Classes,1,Has a sixth form,2,Girls,0,Does not apply,Does not apply,0000,Not applicable,4,Non-selective,1164.0,2,No Special Classes,18-01-2024,1218.0,66.0,1152.0,35.8,0,Not applicable,NaN,NaN,Not applicable,NaN,Not under a feder

In [49]:
# this shows 127 schools missing email, but vast majority do have email address
revised_df["SchoolWebsite"].isna().sum()

127

In [51]:
# dropping the 127 rows with missing school website 
new_df = revised_df.dropna(subset=["SchoolWebsite"])

print(new_df["SchoolWebsite"].isna().sum())  # should print 0

0


As this is a web scrapping task, we'll now import BeautifulSoup to help scrape emails from the school websites!

In [57]:
# because this could take a long time, we first want to test our method with the first 20 websites. Therefore, we'll take first 20 schools to scrape. 
first_20_schools_df = new_df[:20].copy() # USING .copy() HERE is very important to avoid error after executing code block below. 
first_20_schools_df.shape

(20, 135)

In [58]:
# This lambda expression reformats emails that don't have 'https://' at start, so that BeautifulSoup can successfully land on website to search for email address. 
first_20_schools_df["SchoolWebsite"] = first_20_schools_df["SchoolWebsite"].apply(
    lambda url: f"https://{url}" if not url.startswith(("http://", "https://")) else url)
# from my research, I've learnt even if the website starts with 'http', 'https' should redirect to the website!

In [59]:
import requests
from bs4 import BeautifulSoup
import re

def extract_email_from_url(url):
    try:
        # sends a GET request to the URL
        response = requests.get(url)
        if response.status_code == 200:
            # parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # search for email addresses using a regex pattern
            email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
            emails = re.findall(email_pattern, soup.get_text())

            # return the first found email or None if no email found
            return emails[0] if emails else None
        else:
            return None
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# assuming you've already defined the `extract_email_from_url` function and applied it to extract emails

# ensure `SchoolEmail` is added to the DataFrame
first_20_schools_df['SchoolEmail'] = first_20_schools_df['SchoolWebsite'].apply(lambda x: extract_email_from_url(x) if isinstance(x, str) else None)

# now, check that both columns exist in the DataFrame
print(first_20_schools_df[['SchoolWebsite', 'SchoolEmail']])

                                   SchoolWebsite                             SchoolEmail
49          http://www.haverstock.camden.sch.uk/              office@h3federation.org.uk
50      http://www.parliamenthill.camden.sch.uk/  enquiries@parliamenthill.camden.sch.uk
51           http://www.regenthighschool.org.uk/       enquiries@regenthighschool.org.uk
52            http://www.hampsteadschool.org.uk/        enquiries@hampsteadschool.org.uk
53      http://www.aclandburghley.camden.sch.uk/       info@aclandburghley.camden.sch.uk
54          http://www.camdengirls.camden.sch.uk                                    None
55        http://www.mariafidelis.camden.sch.uk/                                    None
56        http://www.williamellis.camden.sch.uk/                                    None
58              http://www.lasainteunion.org.uk/               general@lsu.camden.sch.uk
170           https://www.koinoniafederation.com              sao@koinoniafederation.com
181                  

We'll now re-run BeautifulSoup, this time using a depth of 5 to see if we can get more emails from the first 20 websites!

In [60]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, urlparse

# initialise a set to track visited URLs to prevent revisiting
visited_urls = set()

def extract_email_from_url(url, depth=1, max_depth=5):
    if depth > max_depth:
        return None
    
    try:
        # send a GET request to the URL
        response = requests.get(url)
        if response.status_code == 200:
            # parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # search for email addresses using a regex pattern
            email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
            emails = re.findall(email_pattern, soup.get_text())

            # if an email is found, return the first one
            if emails:
                return emails[0]
            
            # if no email found, try to find links to crawl (recursive)
            links = soup.find_all('a', href=True)
            for link in links:
                link_url = urljoin(url, link['href'])  # join relative URL with base URL
                if link_url not in visited_urls and urlparse(link_url).netloc == urlparse(url).netloc:  # same domain
                    visited_urls.add(link_url)
                    result = extract_email_from_url(link_url, depth + 1, max_depth)
                    if result:  # if email is found in one of the links, return it
                        return result
        else:
            return None
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# apply the function to the DataFrame, ensuring the URL is a string
first_20_schools_df['SchoolEmail'] = first_20_schools_df['SchoolWebsite'].apply(
    lambda x: extract_email_from_url(x, depth=1, max_depth=5) if isinstance(x, str) else None
)

# check the DataFrame to see the results
print(first_20_schools_df[['SchoolWebsite', 'SchoolEmail']])

                                   SchoolWebsite                             SchoolEmail
49          http://www.haverstock.camden.sch.uk/              office@h3federation.org.uk
50      http://www.parliamenthill.camden.sch.uk/  enquiries@parliamenthill.camden.sch.uk
51           http://www.regenthighschool.org.uk/       enquiries@regenthighschool.org.uk
52            http://www.hampsteadschool.org.uk/        enquiries@hampsteadschool.org.uk
53      http://www.aclandburghley.camden.sch.uk/       info@aclandburghley.camden.sch.uk
54          http://www.camdengirls.camden.sch.uk                                    None
55        http://www.mariafidelis.camden.sch.uk/                                    None
56        http://www.williamellis.camden.sch.uk/                williamellispa@gmail.com
58              http://www.lasainteunion.org.uk/               general@lsu.camden.sch.uk
170           https://www.koinoniafederation.com              sao@koinoniafederation.com
181                  

Enhancing the depth of our search to 5 has given us a reasonable amount of success from our trial batch of 20 schools. Originally, BeautifulSoup was able to attain 13/20 email addresses. However, now we've increased search to depth of 5, we've successsfully attained 16/20!

# Step 3: Contacting the schools we've managed to attain email addresses for

In [74]:
# create dummy dataset 
dummy_email_df = {'SchoolName': ['Hampstead School', "Islington Academy", "Camden Secondary School"],
        'ContactName': ['Nielsen Reaveley', "John Adams", "David Read"],
        'SchoolEmail': ['nareaveley@gmail.com', "john.adams@hotmail.com", "nareaveley@hotmail.com"]}  

dummy_email_df = pd.DataFrame(dummy_email_df)

print(dummy_email_df)

                SchoolName       ContactName             SchoolEmail
0         Hampstead School  Nielsen Reaveley    nareaveley@gmail.com
1        Islington Academy        John Adams  john.adams@hotmail.com
2  Camden Secondary School        David Read  nareaveley@hotmail.com


In [75]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# define your email credentials
sender_email = "nareaveley@gmail.com"
sender_password = "hlbz vrrt xnyt vxmz" # password generated from google 'app passwords' page. 

# email list (ensure this is a list of email addresses, not a DataFrame column)
email_list = dummy_email_df["SchoolEmail"].tolist()  # convert to list if needed

# set up the server (Gmail in this example)
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()

# log in to your email account
server.login(sender_email, sender_password)

# send the email to each recipient in the list
for recipient_email in email_list:
    try:
        # gets the school name from the DataFrame 
        school_name = dummy_email_df.loc[dummy_email_df["SchoolEmail"] == recipient_email, "SchoolName"].values[0]

        # set up the MIME
        subject = "Introducing EddyAI"
        body = f"""
        Dear {school_name}, 

        We hope this email finds you well. We would like to introduce you to EddyAI, a solution designed to help schools like {school_name} optimise their operations.
        We believe EddyAI could be a great fit for your needs and would love to discuss the technology further with you.

        Best regards,
        Nielsen 
        EddyAI Team
        """
        
        # create the email message
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = recipient_email
        message["Subject"] = subject
        message.attach(MIMEText(body, "plain"))

        # send the email
        server.sendmail(sender_email, recipient_email, message.as_string())
        print(f"Email sent to {recipient_email}")
    
    except Exception as e:
        print(f"Error sending email to {recipient_email}: {e}")

# close the server connection
server.quit()

Email sent to nareaveley@gmail.com
Email sent to john.adams@hotmail.com
Email sent to nareaveley@hotmail.com


(221,
 b'2.0.0 closing connection 5b1f17b1804b1-43aba58760bsm96243055e9.38 - gsmtp')

Obviously, we would replace our dummy dataset with our actual dataframe if we were sending emails to prospective clients. Secondly, we could further personalise the emails and massively improve the quality of the content of the emails!